# Data preparation for Session KNN Algo on the Coveo Data

### Import libraries 

In [ ]:
#import libraries 
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [ ]:
#import other lib 
import os 
import time
import datetime 
from tqdm import tqdm 
from collections import Counter

### load, Analyse and prepare data

In [ ]:
#import the sample browsing data 

browsing_train = pd.read_csv('browsing_product_only.csv')

In [ ]:
browsing_train

In [ ]:
# convert timestamp to time 

def convert_to_time(df):
    
    df['time'] = (df['server_timestamp_epoch_ms'] / 1000).astype(int)
    #df['time'] = pd.to_datetime(df['time'].astype(int), unit='s')
    
    return df

In [ ]:
convert_to_time(browsing_train)

In [ ]:
browsing_train.columns

In [ ]:
browsing_train.dtypes

In [ ]:
"""
The object datatype is a string or a mixed data type. 

Thus, I need to make the session_id_hash and the product_sku_hash to int64. 

"""

In [ ]:
# session_key, item_key, time_key

browsing_session = browsing_train[['session_id_hash', 'product_sku_hash', 'time']] 

In [ ]:
browsing_session.dtypes

In [ ]:
browsing_session.info()

In [ ]:
browsing_session.head()

In [ ]:
print(browsing_session['session_id_hash'].nunique())

In [ ]:
3280584 + 1

In [ ]:
SessionId = list(range(1,3280585))

In [ ]:
print(len(SessionId))
print(type(SessionId))


In [ ]:
session_list = list(browsing_session['session_id_hash'].unique())

In [ ]:
print(len(session_list))
print(type(session_list))

In [ ]:
Session_Id_df = pd.DataFrame(SessionId, columns=['SessionId'])
Session_Id_df

In [ ]:
session_list_df = pd.DataFrame(session_list, columns=['session_id_hash'])
session_list_df

In [ ]:
new_session_list = pd.concat([session_list_df, Session_Id_df], axis=1)

In [ ]:
new_session_list

In [ ]:
browsing_session

In [ ]:
browsing_session1 = pd.merge(browsing_session, new_session_list, on='session_id_hash')

In [ ]:
browsing_session1

In [ ]:
print(browsing_session1.nunique())
print(browsing_session1.isnull().sum())

In [ ]:
browsing_session1.dtypes

In [ ]:
browsing_session1

In [ ]:
product_list = list(browsing_session1['product_sku_hash'].unique())
product_list_df = pd.DataFrame(product_list, columns=['product_sku_hash'])
product_list_df

In [ ]:
print(len(browsing_session1['product_sku_hash'].unique()))

In [ ]:
10000+57483

In [ ]:
item_list = list(range(10000,67483))
print(len(item_list))

In [ ]:
ItemId_list_df = pd.DataFrame(item_list, columns=['ItemId'])
ItemId_list_df

In [ ]:
new_product_list = pd.concat([product_list_df, ItemId_list_df], axis=1)

In [ ]:
new_product_list

In [ ]:
print(new_product_list.isnull().sum())

In [ ]:
browsing_session2 = pd.merge(browsing_session1, new_product_list, on='product_sku_hash')

In [ ]:
browsing_session2

In [ ]:
print(browsing_session2.nunique())

In [ ]:
print(browsing_session2.isnull().sum())

In [ ]:
browsing_session2.dtypes

In [ ]:
print(len(browsing_train))
print(len(browsing_session2))

In [ ]:
main_df = browsing_session2[['SessionId', 'ItemId', 'time']]

In [ ]:
main_df

In [ ]:
print(main_df.isnull().sum())
print('--------------------')
print(main_df.dtypes)

In [ ]:
#rename columns 

col1 = ['session_id', 'item_id', 'ts']

main_df.columns = col1

In [ ]:
main_df.head()

In [ ]:
product_count = main_df['item_id'].value_counts().rename_axis('item_id').reset_index(name='counts')

In [ ]:
product_count

In [ ]:
print(product_count.nunique())
print(product_count.isnull().sum())

In [ ]:
product_count

In [ ]:
main_df2 = pd.merge(main_df, product_count, on='item_id')

In [ ]:
main_df2

In [ ]:
print(main_df2.nunique())
print(main_df2.isnull().sum())

In [ ]:
main_df2['counts'].max()

In [ ]:
main_df2['counts'].min()

In [ ]:
"""
So, basically, I want to remove all sessions with products that appears less than 4 times. 

Sessions

"""

In [ ]:
main_df2

In [ ]:
sessions_to_remove = list(main_df2.loc[main_df2['counts']<6, 'session_id'])

In [ ]:
sessions_to_remove

In [ ]:
len(sessions_to_remove)

In [ ]:
tmp = main_df2[main_df2.session_id.isin(sessions_to_remove)]

In [ ]:
tmp

In [ ]:
tmp.session_id.nunique()

In [ ]:
main_df3 = main_df2[main_df2.session_id.isin(sessions_to_remove) == False]

In [ ]:
main_df3 = main_df3.reset_index(drop=True)
main_df3

In [ ]:
main_df3['counts'].min()

In [ ]:
main_df3['counts'].max()

In [ ]:
main_df3.columns

In [ ]:
train_df = main_df3[['session_id', 'item_id', 'ts', 'counts']]

In [ ]:
train_df

In [ ]:
train_df['counts'].min()

In [ ]:
train_df = train_df[['session_id', 'item_id', 'ts']]

In [ ]:
train_df

In [ ]:
print(train_df.nunique())
print(train_df.isnull().sum())
print(train_df.dtypes)

In [ ]:
def preprocess_data1(df, min_session_count=3):

    df.dropna(inplace=True)
    item_counts = df.groupby(["session_id"]).count()["item_id"]
    df = df[df["session_id"].isin(item_counts[item_counts >= min_session_count].index)].reset_index(drop=True)
    
    return df

In [ ]:
train_df = preprocess_data1(train_df)
train_df

In [ ]:
print(train_df.nunique())
print(train_df.isnull().sum())
print(train_df.dtypes)

In [ ]:
"""
This should be correct, I think 

"""

In [ ]:
train_df

In [ ]:
train_df.to_csv('SessionKNN_train.csv', index=False)

In [ ]:
test = pd.read_csv('SessionKNN_train.csv')

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test.nunique()